In [1]:
import pandas as pd
import numpy as np


path = 'data/Housing_Price_Data.csv'
data = pd.read_csv(path)
for column in data.select_dtypes(include='object').columns:
    if data[column].nunique() == 2 and set(data[column].unique()).issubset({'yes', 'no'}):
        data[column] = data[column].map({'yes': 1, 'no': 0})


data = data.drop(['furnishingstatus'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   price            545 non-null    int64
 1   area             545 non-null    int64
 2   bedrooms         545 non-null    int64
 3   bathrooms        545 non-null    int64
 4   stories          545 non-null    int64
 5   mainroad         545 non-null    int64
 6   guestroom        545 non-null    int64
 7   basement         545 non-null    int64
 8   hotwaterheating  545 non-null    int64
 9   airconditioning  545 non-null    int64
 10  parking          545 non-null    int64
 11  prefarea         545 non-null    int64
dtypes: int64(12)
memory usage: 51.2 KB


In [2]:
data.describe()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000,545.000000,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.858716,0.177982,0.350459,0.045872,0.315596,0.693578,0.234862
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.348635,0.382849,0.477552,0.209399,0.465180,0.861586,0.424302
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000


In [3]:
from PCA import PCA

y = data['price'].values
X = data.drop(['price'], axis=1).values


In [4]:
pca = PCA(X)
n = 10
# Используем основной метод PCA
transformed_data_svd = pca.pca_qr(n_components=n)
print("Transformed Data (SVD-based PCA):")
print(transformed_data_svd)

# Используем асимптотически быстрый метод PCA
transformed_data_fast = pca.fast_pca(n_components=n)
print("\nTransformed Data (Fast PCA):")
print(transformed_data_fast)

# Используем Kernel PCA с RBF ядром
transformed_data_kernel = pca.kernel_pca(n_components=n)
print("\nTransformed Data (Kernel PCA with RBF kernel):")
print(transformed_data_kernel)

Transformed Data (SVD-based PCA):
(array([[ 2.26945905e+03,  1.77812173e+00,  7.35254019e-01, ...,
        -7.81947848e-01,  6.43376912e-02,  1.39551780e-01],
       [ 3.80945925e+03,  3.04146503e+00,  1.34358854e+00, ...,
        -3.54659897e-01,  1.30087291e-01, -8.47367276e-02],
       [ 4.80945889e+03,  3.12263262e-02,  6.60673977e-01, ...,
        -3.76996047e-01,  5.62378733e-01, -1.20731689e-02],
       ...,
       [-1.53054147e+03, -1.21464252e+00, -3.43642544e-01, ...,
        -1.03336816e-01, -1.98564316e-01, -1.82718700e-01],
       [-2.24054146e+03, -6.40546775e-01, -2.67204014e-01, ...,
         5.92509160e-03,  1.94719340e-01,  5.71335012e-01],
       [-1.30054139e+03,  7.02358569e-02, -5.84645503e-01, ...,
         3.22101113e-02, -1.57757319e-01, -1.91448708e-01]]), array([4.70951221e+06, 9.92425022e-01, 6.54101710e-01, 4.02844475e-01,
       2.73309134e-01, 1.88658678e-01, 1.74389374e-01, 1.45770170e-01,
       1.02292162e-01, 9.88981056e-02]), array([[ 9.99999984e-01,

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train,  y_test = train_test_split(X,y, train_size=0.8, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
res0 = mean_squared_error(model.predict(X_test), y_test)
print(res0)


1800793719718.9675


In [6]:
X_train, X_test, y_train,  y_test = train_test_split(transformed_data_svd[0], y, train_size=0.8, random_state=42)
model1 = LinearRegression()
model1.fit(X_train, y_train)
res1 = mean_squared_error(model1.predict(X_test), y_test)
print(res1)

1877232800627.9111


In [7]:
X_train, X_test, y_train,  y_test = train_test_split(transformed_data_fast[0], y, train_size=0.8, random_state=42)
model2 = LinearRegression()
model2.fit(X_train, y_train)
res2 = mean_squared_error(model2.predict(X_test), y_test)
print(res2)

1877232800628.6304


In [8]:
X_train, X_test, y_train,  y_test = train_test_split(transformed_data_kernel[0], y, train_size=0.8, random_state=42)
model3 = LinearRegression()
model3.fit(X_train, y_train)
res3 = mean_squared_error(model3.predict(X_test), y_test)
print(res3)

4545339412401.969
